In [ ]:
!pip install seaborn

In [ ]:
!pip install vectorbt

In [1]:
import vectorbt as vbt
import yfinance as yf
import talib as ta
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# --- データ取得 ---
price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]  # マルチインデックス対策

# --- RSI計算 ---
n1, n2 = 14, 28
rsiS = ta.RSI(price, timeperiod=n1)
rsiL = ta.RSI(price, timeperiod=n2)

# --- シグナル ---
entries = rsiS > rsiL
exits = rsiS < rsiL
entries = entries.fillna(False)
exits = exits.fillna(False)

# --- バックテスト実行 ---
pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

# --- チャート構築（2段構え） ---
main_fig = pf.plot()  # vectorbt標準チャート
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    row_heights=[0.7, 0.3], vertical_spacing=0.05,
                    subplot_titles=("Price + Trades", "RSI"))

# メインチャート追加（上段）
for trace in main_fig.data:
    fig.add_trace(trace, row=1, col=1)

# RSI追加（下段）
fig.add_trace(go.Scatter(x=price.index, y=rsiS, mode='lines', name=f'RSI {n1}', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=price.index, y=rsiL, mode='lines', name=f'RSI {n2}', line=dict(color='red')), row=2, col=1)

# 軸設定
fig.update_yaxes(title_text="RSI", range=[0, 100], row=2, col=1)
fig.update_layout(height=800, showlegend=True, title_text="RSI Cross Strategy (Jupyter表示対応)")

# ✅ ここでJupyterに直接表示（iframeレンダラー使用）
fig.show()


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_75622/2076355835.py:9: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')


In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_strategy_with_indicator(pf, price, indicators: dict, title="Strategy Result"):
    main_fig = pf.plot()
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True,
        row_heights=[0.7, 0.3], vertical_spacing=0.05,
        subplot_titles=("Price + Trades", "Indicators")
    )

    # 上段：トレード結果
    for trace in main_fig.data:
        fig.add_trace(trace, row=1, col=1)

    # 下段：指標（複数対応）
    for name, series in indicators.items():
        fig.add_trace(go.Scatter(
            x=price.index, y=series, mode='lines', name=name
        ), row=2, col=1)

    fig.update_layout(height=800, title_text=title, showlegend=True)
    return fig


In [3]:
indicators = {
    f"RSI {n1}": rsiS,
    f"RSI {n2}": rsiL
}
fig = plot_strategy_with_indicator(pf, price, indicators, title="RSI Cross Strategy")
fig.show()


In [4]:
maS = price.rolling(5).mean()
maL = price.rolling(25).mean()
entries = maS > maL
exits = maS < maL

pf = vbt.Portfolio.from_signals(price, entries, exits)

indicators = {
    "MA 5": maS,
    "MA 25": maL
}
fig = plot_strategy_with_indicator(pf, price, indicators, title="MA Cross Strategy")
fig.show()

In [29]:
import vectorbt as vbt
import yfinance as yf
import talib as ta
import pandas as pd
import plotly.graph_objects as go

# --- データ取得 ---
price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]

# --- RSI計算 ---
n1, n2 = 14, 28
rsiS = ta.RSI(price, timeperiod=n1)
rsiL = ta.RSI(price, timeperiod=n2)

# --- シグナル作成 ---
entries = rsiS > rsiL
exits = rsiS < rsiL
entries = entries.fillna(False)
exits = exits.fillna(False)

# --- バックテスト ---
pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

# --- 通常チャート取得 ---
fig = pf.plot()
"""
# --- RSI を上段 Price チャートにオーバーレイ（右軸）---
fig.add_trace(go.Scatter(
    x=price.index, y=rsiS, mode='lines', name=f'RSI {n1}',
    yaxis='y5', line=dict(color='blue', width=1)
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=price.index, y=rsiL, mode='lines', name=f'RSI {n2}',
    yaxis='y5', line=dict(color='red', width=1)
), row=1, col=1)

# --- 右側 RSI 用 Y軸追加 ---
fig.update_layout(
    yaxis5=dict(
        overlaying='y',
        side='right',
        title='RSI',
        range=[0, 100],
        showgrid=False
    )
)

# --- レイアウト調整（タイトル・凡例） ---
fig.update_layout(
    height=950,
    title_text="RSI Cross Strategy with Unified Layout",
    title_x=0.5,
    margin=dict(t=80),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="center",
        x=0.5,
        title_text="",
        font=dict(size=10),
        bgcolor="rgba(255,255,255,0)"
    )
)
"""
fig.show()


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_75622/637330801.py:8: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.



In [35]:
import vectorbt as vbt
import yfinance as yf
import talib as ta
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- データ取得 ---
price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]

# --- RSI計算 ---
n1, n2 = 14, 28
rsiS = ta.RSI(price, timeperiod=n1)
rsiL = ta.RSI(price, timeperiod=n2)

# --- シグナル作成 ---
entries = (rsiS > rsiL).fillna(False)
exits = (rsiS < rsiL).fillna(False)

# --- バックテスト ---
pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

# 成績確認
print(pf.stats())

# --- チャート（Jupyter表示用） ---
main_fig = pf.plot()
rsi_trace_1 = go.Scatter(x=rsiS.index, y=rsiS, name=f'RSI {n1}', line=dict(color='blue'))
rsi_trace_2 = go.Scatter(x=rsiL.index, y=rsiL, name=f'RSI {n2}', line=dict(color='red'))

# --- RSI サブプロットを作成 ---
rsi_fig = make_subplots(rows=1, cols=1, subplot_titles=("RSI",))
rsi_fig.add_trace(rsi_trace_1)
rsi_fig.add_trace(rsi_trace_2)
rsi_fig.update_yaxes(title_text="RSI", range=[0, 100])
rsi_fig.update_layout(height=300, margin=dict(t=40, b=40))

# --- 表示（2つのFigureを並べて表示） ---
main_fig.show()
rsi_fig.show()


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_75622/1325563636.py:9: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/pytho

Start                         2021-01-03 15:00:00+00:00
End                           2022-03-29 15:00:00+00:00
Period                                              303
Start Value                                     10000.0
End Value                                  10979.757919
Total Return [%]                               9.797579
Benchmark Return [%]                          45.648884
Max Gross Exposure [%]                            100.0
Total Fees Paid                              795.471679
Max Drawdown [%]                              11.958525
Max Drawdown Duration                             139.0
Total Trades                                         21
Total Closed Trades                                  20
Total Open Trades                                     1
Open Trade PnL                              1228.503182
Win Rate [%]                                       25.0
Best Trade [%]                                15.550595
Worst Trade [%]                               -7

In [37]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import talib as ta

# データ取得
price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')

# マルチインデックス対応
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]

# RSIパラメータ範囲
ns_range = list(range(5, 30, 5))   # 短期
nl_range = list(range(10, 55, 5))  # 長期

# 条件：ns < nl
params = [(ns, nl) for ns in ns_range for nl in nl_range if ns < nl]

# 結果格納用
portfolios = {}

# 各パラメータで戦略実行
for ns, nl in params:
    rsiS = ta.RSI(price, timeperiod=ns)
    rsiL = ta.RSI(price, timeperiod=nl)

    entries = (rsiS > rsiL).fillna(False)
    exits = (rsiS < rsiL).fillna(False)

    pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)
    portfolios[(ns, nl)] = pf.stats()['Total Return [%]']

# DataFrame化して表示（行=ns, 列=nl）
result_df = pd.Series(portfolios).unstack()
result_df.style.background_gradient(cmap='RdYlGn')

# 結果を DataFrame に変換
result_df = pd.Series(portfolios).unstack()
result_df

/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_75622/1190906730.py:7: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/pytho

,10,15,20,25,30,35,40,45,50
5,27.444114,25.795018,26.708571,14.359155,11.504934,12.238371,12.238371,13.597852,5.999586
10,NaN,30.157325,32.434118,24.155639,17.763877,24.065689,23.564557,25.061226,11.431632
15,NaN,NaN,35.730389,17.981295,5.665841,5.071808,3.250336,0.282106,-8.506893
20,NaN,NaN,NaN,1.748579,-2.441415,0.984348,1.591875,-0.470215,-15.657336
25,NaN,NaN,NaN,NaN,-1.724887,-9.656692,-13.219245,-7.220724,-11.836218


In [39]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import talib as ta
import plotly.graph_objects as go

# データ取得
price = vbt.YFData.download("7203.T", start="2021-01-01", end="2022-03-31").get("Close")

# マルチインデックス対応（普遍）
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]

# パラメータの組み合わせ
ns_range = list(range(5, 30, 5))
nl_range = list(range(10, 55, 5))
params = [(ns, nl) for ns in ns_range for nl in nl_range if ns < nl]

# 結果格納用
result_table = pd.DataFrame(index=ns_range, columns=nl_range, dtype=float)

# RSI クロス戦略のループ実行
for ns, nl in params:
    rsiS = ta.RSI(price, timeperiod=ns)
    rsiL = ta.RSI(price, timeperiod=nl)
    entries = (rsiS > rsiL).fillna(False)
    exits = (rsiS < rsiL).fillna(False)
    pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)
    result_table.loc[ns, nl] = pf.stats()['Total Return [%]']

# NaNは 0 にしておく（必要に応じて）
result_table = result_table.fillna(0)

# ヒートマップ表示（plotly）
fig = go.Figure(data=go.Heatmap(
    z=result_table.values,
    x=result_table.columns.astype(str),
    y=result_table.index.astype(str),
    colorscale='Viridis',
    colorbar=dict(title="Total Return (%)")
))
fig.update_layout(
    title="RSIクロス戦略のパラメータ最適化",
    xaxis_title="長期 RSI",
    yaxis_title="短期 RSI",
    height=600,
    width=800
)
fig.show()


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_75622/1735283187.py:8: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/pytho

In [47]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import talib as ta
import plotly.graph_objects as go

# データ取得
price = vbt.YFData.download("7203.T", start="2021-01-01", end="2022-03-31").get("Close")
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]

# パラメータ範囲設定
ns_range = list(range(5, 30, 5))     # 短期 RSI
nl_range = list(range(10, 55, 5))    # 長期 RSI
params = [(ns, nl) for ns in ns_range for nl in nl_range if ns < nl]

# 結果格納用 DataFrame
result_table = pd.DataFrame(index=ns_range, columns=nl_range, dtype=float)

# RSIクロス戦略のバックテストループ
for ns, nl in params:
    rsiS = ta.RSI(price, timeperiod=ns)
    rsiL = ta.RSI(price, timeperiod=nl)
    entries = (rsiS > rsiL).fillna(False)
    exits = (rsiS < rsiL).fillna(False)
    pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)
    result_table.loc[ns, nl] = pf.stats()['Total Return [%]']

# NaNは 0 に
result_table = result_table.fillna(0)

# ✅ Plotly ヒートマップ表示（RdYlGnで backtrader風に）
fig = go.Figure(data=go.Heatmap(
    z=result_table.values,
    x=[f"長期:{col}" for col in result_table.columns],
    y=[f"短期:{idx}" for idx in result_table.index],
    colorscale='RdYlGn',
    colorbar=dict(title="Total Return [%]")
))
fig.update_layout(
    title="RSIクロス戦略のパラメータ最適化（Total Return [%]）",
    xaxis_title="長期 RSI",
    yaxis_title="短期 RSI",
    width=900,
    height=600
)
fig.show()


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_75622/543241605.py:8: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python

In [53]:
# ハイライトしたい RSI パラメータ
highlight_ns = 14
highlight_nl = 28

# 列名・行名をコピーして編集
renamed_columns = [f"{nl} ←" if nl == highlight_nl else str(nl) for nl in result_df.columns]
renamed_index = [f"{ns} ↑" if ns == highlight_ns else str(ns) for ns in result_df.index]

# インデックスと列名を差し替え
result_df.index = renamed_index
result_df.columns = renamed_columns

# スタイル付きで表示
styled = result_df.style.format("{:.2f}") \
    .background_gradient(cmap='RdYlGn', axis=None) \
    .set_caption("RSIクロス戦略のパラメータ最適化（Total Return [%]）")

styled

,RSI L=10,RSI L=15,RSI L=20,RSI L=25,RSI L=30,RSI L=35,RSI L=40,RSI L=45,RSI L=50
RSI S=5,27.44,25.80,26.71,14.36,11.50,12.24,12.24,13.60,6.00
RSI S=10,0.00,30.16,32.43,24.16,17.76,24.07,23.56,25.06,11.43
RSI S=15,0.00,0.00,35.73,17.98,5.67,5.07,3.25,0.28,-8.51
RSI S=20,0.00,0.00,0.00,1.75,-2.44,0.98,1.59,-0.47,-15.66
RSI S=25,0.00,0.00,0.00,0.00,-1.72,-9.66,-13.22,-7.22,-11.84


In [54]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import talib as ta
import plotly.graph_objects as go

# データ取得
price = vbt.YFData.download("7203.T", start="2021-01-01", end="2022-03-31").get("Close")
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]

# パラメータ範囲設定
ns_range = list(range(5, 30, 5))     # 短期 RSI
nl_range = list(range(10, 55, 5))    # 長期 RSI
params = [(ns, nl) for ns in ns_range for nl in nl_range if ns < nl]

# 結果格納用 DataFrame
result_table = pd.DataFrame(index=ns_range, columns=nl_range, dtype=float)

# RSIクロス戦略のバックテストループ
for ns, nl in params:
    rsiS = ta.RSI(price, timeperiod=ns)
    rsiL = ta.RSI(price, timeperiod=nl)
    entries = (rsiS > rsiL).fillna(False)
    exits = (rsiS < rsiL).fillna(False)
    pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)
    result_table.loc[ns, nl] = pf.stats()['Total Return [%]']

# NaNは 0 に
result_table = result_table.fillna(0)

# ✅ Plotly ヒートマップ表示（セルに数値を表示）
fig = go.Figure(data=go.Heatmap(
    z=result_table.values,
    x=[f"{col}" for col in result_table.columns],
    y=[f"{idx}" for idx in result_table.index],
    colorscale='RdYlGn',
    colorbar=dict(title="Total Return [%]"),
    text=result_table.round(2).astype(str),          # ⬅ セル内の文字
    texttemplate="%{text}",                          # ⬅ 表示フォーマット
    hovertemplate="短期 RSI=%{y}<br>長期 RSI=%{x}<br>リターン=%{z:.2f}%<extra></extra>"
))
fig.update_layout(
    title="RSIクロス戦略のパラメータ最適化（Total Return [%]）",
    xaxis_title="長期 RSI",
    yaxis_title="短期 RSI",
    width=900,
    height=600
)
fig.show()


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_75622/3034826040.py:8: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.2/lib/pytho

In [55]:
# ベストペア

best_ns, best_nl = result_df.stack().idxmax()
best_return = result_df.stack().max()

print(f"🔥 最も良い組み合わせ: ns={best_ns}, nl={best_nl}（リターン: {best_return:.2f}%）")

🔥 最も良い組み合わせ: ns=RSI S=15, nl=RSI L=20（リターン: 35.73%）
